Todas las dependencias que tueve que ir instalando las fuí comentado para que quede registrado
Funcione con python 3.11 porque knowledge_graph_maker solo funciona con esa versión de python.

In [80]:
#!pip install knowledge-graph-maker

In [81]:
#!pip install python-dotenv

In [82]:
#%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

In [1]:
#Tuve que abrir una cuenta en Groq para obtener una API KEY
import os

os.environ["GROQ_API_KEY"] = "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"

In [2]:
from knowledge_graph_maker import GraphMaker, Ontology, GroqClient
from knowledge_graph_maker import Document

In [85]:
#!pip install langchain

In [86]:
#!pip install langchain-community

In [87]:
#!pip install pypdf

In [28]:
#Esta parte es donde se pone a mano el pdf que se quiere analizar, esto hay que hacerlo con fasAPI 
#para que reciba con el verbo POST el pdf por el body.

from langchain.document_loaders import PyPDFLoader
from typing import List

def scrap_pdf(pdf_path: str) -> List[str]:
    # Paso 1: Cargar el PDF
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    
    # Paso 2: Extraer el texto de cada página y almacenarlo en una lista
    text_list = [page.page_content for page in pages]
    
    return text_list

# Uso de la función
#pdf_path = "http://www.fiumsa.edu.bo/docentes/mramirez/capitulo_II.pdf"
pdf_path = "https://www.williammurdoch.bham.sch.uk/admin/ckfinder/userfiles/files/Y5%20Reading%20Task%20-%20Elizabeth%20I.pdf"
#pdf_path = "https://cms.fi.uba.ar/uploads/Informacion_sobre_Doble_Diploma_docx_4eb4306159.pdf"
scraped_text = scrap_pdf(pdf_path)

In [29]:
#La ontología, se tiene que poner como archivo, en principo ponemos el texto en chatGPT y pedimos que nos genere una ontología
#debe pasarse el json en forma por body en un POST a la API de fasAPI.
ontology = Ontology(
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        "Place",
        "Document",
        "Organisation",
        "Action",
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)

'''ontology = Ontology(
    labels=[
        {"Person": "Nombres de personas, incluyendo estudiantes, profesores y autoridades."},
        {"Program": "Nombres de programas educativos, como 'Doble Diploma' o nombres específicos de carreras o disciplinas."},
        {"Institution": "Nombres de instituciones educativas involucradas, como la 'Facultad de Ingeniería', 'Universidad de Buenos Aires', o universidades extranjeras."},
        {"Country": "Nombres de países involucrados en los programas, como 'Argentina' o cualquier país extranjero mencionado."},
        {"Document": "Documentos formales o referencias a normativas, acuerdos, o reglamentos."},
        {"Date": "Fechas importantes mencionadas en el contexto del programa, como fechas límite o de inicio de actividades."},
        {"Requirement": "Criterios o condiciones que los estudiantes deben cumplir para participar en el programa."},
        {"Procedure": "Pasos o procesos que los estudiantes deben seguir para aplicar o completar el programa de Doble Diploma."},
        {"Miscellaneous": "Cualquier otro concepto relevante que no se categorice bajo las etiquetas anteriores, como términos específicos del programa o conceptos relacionados a intercambios académicos."},
    ],
    relationships=[
        "Association between a Person and a Program",
        "Connection between an Institution and a Program",
        "Link between a Country and an Institution",
        "Requirement or condition related to a Program",
        "Procedure or step involved in the application or completion of the Program",
    ],
)'''


'ontology = Ontology(\n    labels=[\n        {"Person": "Nombres de personas, incluyendo estudiantes, profesores y autoridades."},\n        {"Program": "Nombres de programas educativos, como \'Doble Diploma\' o nombres específicos de carreras o disciplinas."},\n        {"Institution": "Nombres de instituciones educativas involucradas, como la \'Facultad de Ingeniería\', \'Universidad de Buenos Aires\', o universidades extranjeras."},\n        {"Country": "Nombres de países involucrados en los programas, como \'Argentina\' o cualquier país extranjero mencionado."},\n        {"Document": "Documentos formales o referencias a normativas, acuerdos, o reglamentos."},\n        {"Date": "Fechas importantes mencionadas en el contexto del programa, como fechas límite o de inicio de actividades."},\n        {"Requirement": "Criterios o condiciones que los estudiantes deben cumplir para participar en el programa."},\n        {"Procedure": "Pasos o procesos que los estudiantes deben seguir para apl

In [30]:
## Groq models. Se puede probar con el que mejor funcione, hay que probar.
model = "mixtral-8x7b-32768"
#model="gemma-7b-it"

## Open AI models
##oai_model="gpt-3.5-turbo"

## Use Groq
llm = GroqClient(model=model, temperature=0.1, top_p=0.5)
## OR Use OpenAI
##llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)

In [31]:
#Esto es interno, acá se genera el grafo
import datetime
current_time = str(datetime.datetime.now())
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary

docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    scraped_text
)

In [32]:
graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=0 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  mixtral-8x7b-32768


...

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 20.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-28 13:07:53 - INFO 
LLM Response:
[
{
"node_1": {"label": "Person", "name": "Queen Elizabeth I"},
"node_2": {"label": "Place", "name": "port of Tilbury"},
"relationship": "Queen Elizabeth I went to the port of Tilbury"},
{
"node_1": {"label": "Person", "name": "Queen Elizabeth I"},
"node_2": {"label": "Miscellaneous", "name": "time of the Armada"},
"relationship": "The event occurred during the time of the Armada and Queen Elizabeth I was present"},
{
"node_1": {"label": "Person", "name": "Queen Elizabeth I"},
"node_2": {"label": "Event", "name": "addressing her troops"},
"relationship": "Queen Elizabeth I addressed her troops at the port of Tilbury"},
{
"nod

Using Model:  mixtral-8x7b-32768


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 26.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-28 13:08:21 - INFO 
LLM Response:
[
{"node_1": {"label": "Person", "name": "Queen Elizabeth I"},
"node_2": {"label": "Miscellaneous", "name": "dynasty"},
"relationship": "Queen Elizabeth I is a part of a dynasty, as suggested by the context in which the word 'dynasty' is used."},
{"node_1": {"label": "Miscellaneous", "name": "dynasty"},
"node_2": {"label": "Miscellaneous", "name": "sequencing"},
"relationship": "The concept of 'dynasty' is being explained in the context of sequencing, suggesting a relationship between the two."},
{"node_1": {"label": "Person", "name": "Queen Elizabeth I"},
"node_2": {"label": "Place", "name": "England"},
"relationship": "Quee

Total number of Edges 76


In [33]:
#Este no hace falta en el archivo, es para ver el grafo en consola.
for edge in graph:
    print(edge.model_dump(exclude=['metadata']), "\n\n")

{'node_1': {'label': 'Person', 'name': 'Queen Elizabeth I'}, 'node_2': {'label': 'Person', 'name': 'Henry VIII'}, 'relationship': 'Queen Elizabeth I is the daughter of Henry VIII', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Queen Elizabeth I'}, 'node_2': {'label': 'Person', 'name': 'Anne Boleyn'}, 'relationship': 'Queen Elizabeth I is the daughter of Anne Boleyn', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Queen Elizabeth I'}, 'node_2': {'label': 'Object', 'name': 'England'}, 'relationship': 'Queen Elizabeth I was the Queen of England', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Queen Elizabeth I'}, 'node_2': {'label': 'Object', 'name': 'Ireland'}, 'relationship': 'Queen Elizabeth I was the Queen of Ireland', 'order': 0} 


{'node_1': {'label': 'Person', 'name': 'Queen Elizabeth I'}, 'node_2': {'label': 'Number', 'name': '44 years'}, 'relationship': 'Queen Elizabeth I was Queen of England and Ireland for 44 years', 'order': 0} 


{'node_1': {'label':

In [94]:
#!pip install pyvis

In [34]:
#Credenciales de Neo4j
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_URI"]  = os.getenv("NEO4J_URI")
os.environ["AURA_INSTANCEID"]="31995b6a"
os.environ["AURA_INSTANCENAME"]="Instance01"'The Doble Diploma (DD) is a joint engineering degree program between France and a specific institution, such as FIUBA. To be eligible, students must meet requirements such as having a good academic performance, a minimum level of English and French, and approving at least 60% of their career (excluding the CBC). The application process typically takes place in June-July of the year N-1.'


In [35]:
#Se guarda el grafo en Neo4j
from knowledge_graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)

neo4j_graph.save()

INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Statement.CartesianProduct} {category: PERFORMANCE} {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (us))} {position: line: 1, column: 1, offset: 0} for query: 'MATCH (them), (us) WHERE elementId(them)=$them and elementId(us)=$self MERGE(us)-[r:`RELATED` {description: $description, metadata: $metadata, order: $order}]->(them) RETURN r'
INFO:neo4j.notifications:Received notification from DBMS server: {severity: IN

76

In [97]:
#!pip install yfiles_jupyter_graphs

In [36]:
#Este no es es necesario, es para ver el grafo en consola. En la API de backend esto no va.
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
import os

# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [99]:
#%pip install langchain_huggingface --quiet

In [37]:
#Se arman los embeddings de Hugginface

#Esto se tiene que sejecutar solo una vez, si los ebeddings ya estàn creados genera un error
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
import os
from transformers import AutoTokenizer, AutoModel

# Inicializa el modelo y el tokenizador de Hugging Face
model_name = "sentence-transformers/msmarco-distilbert-base-tas-b"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Configura el token de relleno
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Asigna eos_token como pad_token

# Configura las credenciales de Neo4j
url = os.environ.get("NEO4J_URI")
username = os.environ.get("NEO4J_USERNAME")
password = os.environ.get("NEO4J_PASSWORD")

# Verifica si las credenciales están configuradas
if not url or not username or not password:
    raise ValueError("Las credenciales de Neo4j no están configuradas correctamente.")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/msmarco-distilbert-base-tas-b


In [38]:
#Se guardan los embeddings en Neo4j para hacer una búsqueda híbrida.
vector_index = Neo4jVector.from_existing_graph(
    embeddings_model,
    search_type="hybrid",
    url=url,
    username=username,
    password=password,
    node_label="Entity",
    text_node_properties=["name","label"],  # Asegúrate de que esto esté correcto
    embedding_node_property="embedding",
)

In [23]:
#%pip install -U langchain-groq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [39]:
#Hay que analizar si esto es necesario en la api de backend
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# Configuración de Neo4j
graph = Neo4jGraph()
graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

# Configuración del modelo Groq
llm = ChatGroq(
    temperature=0,
    model_name="mixtral-8x7b-32768",  # Asegúrate de usarl
)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

In [40]:
#Hay que ver si esto es necesario en la API de backend
entity_chain.invoke({"question": "Qué requsitos debo cumplir para tener la doble diplomatura?"}).names

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


['Qué',
 'requsitos',
 'debo',
 'cumplir',
 'para',
 'tener',
 'la',
 'doble',
 'diplomatura']

In [41]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]# Configura la conexión a la base de datos Neo4j

    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
        full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes("keyword", $query) YIELD node, score
            WITH node
            CALL {
              WITH node
              MATCH (node)-[r]->(neighbor)
              WHERE NOT type(r) = 'MENTIONS'
              RETURN node.name + ' - ' + type(r) + ' -> ' + neighbor.name AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r]-(neighbor)
              WHERE NOT type(r) = 'MENTIONS'
              RETURN neighbor.name + ' - ' + type(r) + ' -> ' +  node.name AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

# Prueba la función, se puede probar con cualquier pregunta. Esto no va en la API de backend, pero lo de arriba si.
print(structured_retriever("Que es la doble diplomatura?"))

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Doble Diploma - RELATED -> FIUBA
Doble Diploma - RELATED -> Francia
Doble Diploma - RELATED -> Francia
Doble Diploma - RELATED -> École
Doble Diploma - RELATED -> Learning Agreement
Doble Diploma - RELATED -> CBC+3 years approved
Doble Diploma - RELATED -> CBC+2 years approved before applying
Doble Diploma - RELATED -> Annual convocatory
Doble Diploma - RELATED -> ENSTA Bretagne
Doble Diploma - RELATED -> Intercambios Académicos
École - RELATED -> Doble Diploma
Página de Cátedra de Campus del programa - RELATED -> Doble Diploma
Página de la FIUBA - RELATED -> Doble Diploma
Google Form - RELATED -> Doble Diploma
Estudiantes de Ingeniería Naval y Mecánica - RELATED -> Doble Diploma
Doble Diploma (DD) - RELATED -> Francia
Página de la convocatoria - RELATED -> Intercambios Académicos
Página de la FIUBA - RELATED -> Doble Diploma
Doble Diploma de Ingeniería en Francia - RELATED -> FIUBA
Doble Diploma de Ingeniería en Francia - RELATED -> FIUBA
Doble Diploma de Ingeniería en Francia - RELAT

In [42]:
#No va en la API
print(structured_retriever("Qué es la doble diplomatura?"))

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Doble Diploma - RELATED -> FIUBA
Doble Diploma - RELATED -> Francia
Doble Diploma - RELATED -> Francia
Doble Diploma - RELATED -> École
Doble Diploma - RELATED -> Learning Agreement
Doble Diploma - RELATED -> CBC+3 years approved
Doble Diploma - RELATED -> CBC+2 years approved before applying
Doble Diploma - RELATED -> Annual convocatory
Doble Diploma - RELATED -> ENSTA Bretagne
Doble Diploma - RELATED -> Intercambios Académicos
École - RELATED -> Doble Diploma
Página de Cátedra de Campus del programa - RELATED -> Doble Diploma
Página de la FIUBA - RELATED -> Doble Diploma
Google Form - RELATED -> Doble Diploma
Estudiantes de Ingeniería Naval y Mecánica - RELATED -> Doble Diploma
Doble Diploma (DD) - RELATED -> Francia
Página de la FIUBA - RELATED -> Doble Diploma
Página de la convocatoria - RELATED -> Intercambios Académicos
Doble Diploma de Ingeniería en Francia - RELATED -> FIUBA
Doble Diploma de Ingeniería en Francia - RELATED -> FIUBA
Doble Diploma de Ingeniería en Francia - RELAT

In [43]:
#Va en la API es interno.
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = []
    for el in vector_index.similarity_search(question):
        if el.page_content:
            unstructured_data.append(el.page_content)
        else:
            print(f"Warning: Missing or empty text in node {el.metadata['node_id']}")
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ".join(unstructured_data)}
    """
    return final_data


In [44]:
#Va en la API
vector_index = Neo4jVector.from_existing_index(
    index_name="keyword",
    embedding=embeddings_model,  # Añadimos esto
    node_label="Entity",
)

In [45]:
#Historio del chatbot, va en la API.
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.runnables import  RunnableParallel, RunnablePassthrough
from typing import Tuple, List
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser


# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [46]:
#Va en la API
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [47]:
#Esto es necesario para borrar los nodos con ebeddings vacìos. Va en la API porque sino da una falla la query.
# Hay que poner esto para que no existan nulos y se puea hacer la consulta.
# Inicia la conexión con la base de datos
driver = GraphDatabase.driver(url, auth=(username, password))

# Define la función para ejecutar la consulta
def update_entities_text(tx):
    query = """
    MATCH (n:Entity)
    WHERE n.text IS NULL OR n.text = ""
    SET n.text = "Texto predeterminado"
    RETURN n
    """
    result = tx.run(query)
    return [record["n"] for record in result]

# Ejecuta la consulta dentro de una sesión
with driver.session() as session:
    entities = session.write_transaction(update_entities_text)

# Muestra los resultados
for entity in entities:
    print(entity)

# Cierra la conexión
driver.close()


/tmp/ipykernel_9002/3794290000.py:18: DeprecationWarning: write_transaction has been renamed to execute_write
  entities = session.write_transaction(update_entities_text)


<Node element_id='4:e872df9e-93a7-4542-885f-acc50a47928b:75' labels=frozenset({'Entity'}) properties={'name': 'Queen Elizabeth I', 'text': 'Texto predeterminado', 'label': 'Person', 'embedding': [0.224807009100914, -0.05881369486451149, 0.19862693548202515, -0.20296970009803772, 0.1472485363483429, 0.11551161110401154, 0.18553465604782104, -0.06478220969438553, -0.42243829369544983, -0.30021002888679504, -0.21269544959068298, 0.005321940407156944, 0.23076702654361725, -0.015139714814722538, 0.01022057980298996, 0.12571145594120026, -0.07081524282693863, 0.32836875319480896, -0.5136768817901611, -0.08300159871578217, 0.14899545907974243, 0.06925570964813232, -0.04967619851231575, 0.1789713352918625, 0.03334028273820877, 0.09502732753753662, 0.24466852843761444, -0.18526999652385712, -0.15780982375144958, -0.10948344320058823, 0.22432935237884521, -0.07654598355293274, 0.02064700983464718, 0.41167721152305603, -0.1295650601387024, 0.1754942536354065, 0.1446504443883896, -0.12511666119098

In [52]:
#Esta pregunta tiene que entrar por la API de backend usando el verbo POST de fastAPI.
chain.invoke({"question": "Recién aprobé cuatro materias, me puede presentar a la doble diplomatura?"})

Search query: Recién aprobé cuatro materias, me puede presentar a la doble diplomatura?


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'To be eligible for the Double Diploma program after approving four subjects, you need to have approved at least 60% of your career (excluding the CBC), have a good academic performance (above the average of your career, usually more than 7), and meet the language requirements (B2 level of English and B1 level of French). If you meet these requirements, you can apply during the annual call for applications, which usually takes place in June-July of the previous year.'